## Step 1: Generating BLAST initial output with desired template sequences
The sequences chosen for this step has been though literature search, utilising experimentally produced and tested candidates from two different model organisms for this enzyme sysetm, *Acetobacerium dehalogenans* [[1]](https://doi.org/10.1128/JB.01104-08) and *Desulfitobacterium hafinase* [[2]](https://doi.org/10.1111/1574-6941.12433) as template sequences to produce the database. We will be searching for the protein products separately. To keep the markdown short, examples will be given with one protein group at a time (either CP, MT1 or MT2) and the same steps will usually be repeated for the other groups.

Install Biopython if you have not done it before by removing the comment from the first line. To understand how to manipulate the parameters of functions within the common biopython modules such as blast, refer to the biopython cookbook [[3]](http://biopython.org/DIST/docs/tutorial/Tutorial.html).

In [ ]:
#!pip install biopython
import Bio

In [ ]:
#loading modules needed to run and parse BLAST output from within Biopython
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

#defining a function for generating BLAST output from a given fasta file (containing single or multiple accession IDs)

def generate_blast_record():
    fasta_string = input('Enter_path_to_file: ')
    fasta_string_in = open(fasta_string)
    result_handle = NCBIWWW.qblast("blastp", "refseq_protein", fasta_string_in.read(), url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', expect= 1e-10, hitlist_size = 100)
    #please update the parameters as per your requirements
    with open("my_blast_run.xml", "w") as out_handle:
        out_handle.write(result_handle.read())
    result_handle.close()
    result_handle = open("my_blast_run.xml")
    blast_records = NCBIXML.parse(result_handle)
    return(blast_records)

In [ ]:
#input the file and save the handle for vieiwing later
blast_record_list_CP = []
blast_record_list_CP = list(generate_blast_record())
print(blast_record_list_CP)

In [ ]:
# CP blast record dataframe creation by parsing through the output

all_ID_list_CP_demo = []
all_hit_length_list_CP_demo = []
all_hit_evalue_list_CP_demo = []
for blast_record in blast_record_list_CP:
    for alignment in blast_record.alignments:
        all_ID_list_CP_demo.append(alignment.title)
        all_hit_length_list_CP_demo.append(alignment.length)
        for hsp in alignment.hsps:
            all_hit_evalue_list_CP_demo.append(hsp.expect)
print(all_ID_list_CP_demo, all_hit_length_list_CP_demo, all_hit_evalue_list_CP_demo) #checkpoint for loop output

<font color = 'red'> Note: </font> For determining co-localised indices, only a few parameters of the blast output have been imported from the record handle. For a more thorough investigation, the other parameters such as percentage identity, can also be imported.

In [ ]:
#zipping lists together to make the dataframe

import pandas as pd

df_demo_CP = pd.DataFrame(list(zip(all_ID_list_CP_demo, all_hit_length_list_CP_demo, all_hit_evalue_list_CP_demo)), columns = ['CP_description', 'CP_length', 'CP_e-value'])
#some duplicates will also be returned as we have three template sequences that might match differently with the three data
df_demo_CP_no_dups = df_demo_CP.drop_duplicates(subset=['CP_description'])
print(df_demo_CP_no_dups.head())

#sanity check by printing length of both dfs
print(len(df_demo_CP))
print(len(df_demo_CP_no_dups))

<font color = 'red'> Note: </font> Always check for duplicates, especially with multiple queries in blast input as shown here. It is also recommended to create checkpoints at each step so that you have the most latest dataframe in case you need to start over at some point later. This is optional though.

In [ ]:
df_demo_CP.to_csv("ref_db_creation/data/CP_step1.csv", index = False)

## Step 2: Tidying BLAST output and prep for analysis
<br>
<font color = 'red'> Note: </font> The data wrangling demonstrated here is specific to the output generated in Step 1 using the given files. However, it is easily adaptable to any similar dataset or trying to tidy and parse through blast output generated by biopython.

In [ ]:
#we need to split the description column so that it is easy to manipulate for evidence of co-localisation

# 1 : splitting based on '|' character to get 3 columns

df_tidy_CP = pd.DataFrame()
df_tidy_CP[['CP_blastdb_ID', 'CP_ID', 'CP_annotation_full']] = df_master_CP.description_CP.str.split("|", expand = True,)
df_tidy_CP[['CP_length', 'CP_evalue']] = df_master_CP[['length_CP', 'e-value_CP']]
print(df_tidy_CP.head())
print(len(df_tidy_CP)) #keep checking length in tidying steps
df_tidy_CP.head()

In [ ]:
# 2 : Splitting annotation column further

df_tidy_CP[['1', '2']] = df_tidy_CP.CP_annotation_full.str.split('[', expand = True,)
print(df_tidy_CP.head())

<font color = 'red'> Note: </font> It is important to note that the number of columns given for expansion of column based on character separator can be arbitrary, all NCBI records might not have the same format for organism name and therefore this should be carefully evaluated. Usually python will prompt this and return an "unequal" column length error similar to the screenshot below:
<br>
<img src = "images/tidy_step_error.png" width="900" height="1000" align="centre"/>
<br>

In [ ]:
# 3 : Stitching organism name together and final tidying

df_tidy_CP['2'] = df_tidy_CP['2'].str.replace(']','') #cleaning the column
df_tidy_CP = df_tidy_CP.drop('CP_annotation_full', axis = 1) #dropping unwanted columns
df_tidy_CP = df_tidy_CP[['CP_ID', '1', '2', 'CP_length', 'CP_evalue']] #reordering for one last time
desired_column_names_CP = ['CP_ID', 'CP_annotation', 'CP_source_organism', 'CP_length', 'CP_e-value'] #assigning tidy names
df_tidy_CP.columns = desired_column_names_CP
print(df_tidy_CP.isnull().sum()) #checking that there are no null values
df_tidy_CP.head()

In [ ]:
df_tidy_CP_final = df_tidy_CP # assigning a new checkpoint df at end of step 2, good to keep exporting files at different steps
df_tidy_CP_final.to_csv("data/Step_2_CP_demo.csv", index = False) #saving the csv file
df_tidy_CP_final.head()

In [ ]:
#we need to extract headers into a separate csv file for the next step
header = ["CP_ID"]
df_tidy_CP_final.to_csv('all_headers_CP.csv', columns = header, index = False)

The same steps were repeated for the MT1 and MT2 blast output to generate the final list of headers. This header file is the input for the python script to mine the co-localised headers.